In [1]:
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import io

import pyspark.sql.functions as F
import pyspark

from pyspark.sql import SparkSession, Window
from pyspark.conf import SparkConf
# from pyspark.context import SparkContext
from pyspark.sql.types import StringType, ArrayType, StructField, StructType, FloatType, DoubleType, IntegerType

from concurrent.futures import ThreadPoolExecutor

In [2]:
# `sparksession is none: typeerror: 'javapackage' object is not 
# callable` can be raised if the pyspark version being used is 4.0.0
# which is not compatible to a python 3.11.8 version
spark = SparkSession.builder.appName("app").getOrCreate()
    # .config("spark.driver.memory", "14g")\
    # .config("spark.sql.execution.arrow.maxRecordsPerBatch", "100")\
    # .getOrCreate()

In [6]:
# URL = "abfss://{FOLDER_NAME}@sgppipelinesa.dfs.core.windows.net/"
# SILVER_FOLDER_NAME = "sgppipelinesa-silver"
# SUB_FOLDER_NAME = "stage-01"
# SILVER_DATA_PATH = os.path.join(URL.format(FOLDER_NAME=SILVER_FOLDER_NAME), SUB_FOLDER_NAME)
# SILVER_DATA_PATH
# folder_infos = dbutils.fs.ls(BRONZE_DATA_PATH)

DATA_PATH = "../include/data/"
SILVER_FOLDER_NAME = "silver"
SUB_FOLDER_NAME = "stage-01"
SILVER_DATA_PATH = os.path.join(DATA_PATH, os.path.join(SILVER_FOLDER_NAME, SUB_FOLDER_NAME))
SILVER_DATA_PATH

'../include/data/silver\\stage-01'

In [8]:
train_labels_df = spark.read.format("parquet")\
    .load(os.path.join(SILVER_DATA_PATH, "train", "labels.parquet"))

In [ ]:
train_labels_df.show()

+------+--------------------+--------------------+
| value|            filePath|           subjectId|
+------+--------------------+--------------------+
|  male|file:///c:/Users/...|23yipikaye-201008...|
|female|file:///c:/Users/...|Anniepoo-20140308...|
|female|file:///c:/Users/...|Anniepoo-20140308...|
|female|file:///c:/Users/...|Anniepoo-20140308...|
|female|file:///c:/Users/...|Anniepoo-20140308...|
|female|file:///c:/Users/...| 1337ad-20170321-tkg|
|  male|file:///c:/Users/...| 1snoke-20120412-hge|
|  male|file:///c:/Users/...|  Aaron-20080318-kdl|
|  male|file:///c:/Users/...|   1028-20100710-hne|
+------+--------------------+--------------------+



In [ ]:
SPLIT_URL = os.path.join(SILVER_DATA_PATH, "{SPLIT}")
SPLIT_FOLDER = SPLIT_URL.format(SPLIT="train")

# only include the parquet files without labels
train_signal_files = [os.path.join(SPLIT_FOLDER, SIGNAL_DF_FILE) for SIGNAL_DF_FILE in os.listdir(SPLIT_FOLDER) if not "labels" in SIGNAL_DF_FILE]
train_signals_df = spark.read.format("parquet")\
    .load(train_signal_files)

In [25]:
train_signals_df.show()

+--------------------+--------------------+
|           subjectId|             signals|
+--------------------+--------------------+
|23yipikaye-201008...|[0.00491333, 0.00...|
|Anniepoo-20140308...|[-5.187988E-4, -4...|
|Anniepoo-20140308...|[0.012268066, 0.0...|
| 1snoke-20120412-hge|[0.0014343262, 0....|
|Anniepoo-20140308...|[-0.0045166016, -...|
|   1028-20100710-hne|[-0.0054626465, -...|
| 1337ad-20170321-tkg|[0.0015869141, 0....|
|Anniepoo-20140308...|[1.8310547E-4, 7....|
|  Aaron-20080318-kdl|[-2.4414062E-4, -...|
+--------------------+--------------------+



In [30]:
SPLIT_FOLDER = SPLIT_URL.format(SPLIT="validate")

# only include the parquet files without labels
val_signal_files = [os.path.join(SPLIT_FOLDER, SIGNAL_DF_FILE) for SIGNAL_DF_FILE in os.listdir(SPLIT_FOLDER) if not "labels" in SIGNAL_DF_FILE]
val_signals_df = spark.read.format("parquet")\
    .load(val_signal_files)

In [31]:
val_signals_df.show()

+-------------------+--------------------+
|          subjectId|             signals|
+-------------------+--------------------+
| Coren-20141121-pxp|[-3.9672852E-4, 3...|
|1337ad-20170321-ajg|[-0.0010375977, -...|
+-------------------+--------------------+



In [32]:
SPLIT_FOLDER = SPLIT_URL.format(SPLIT="test")

# only include the parquet files without labels
test_signal_files = [os.path.join(SPLIT_FOLDER, SIGNAL_DF_FILE) for SIGNAL_DF_FILE in os.listdir(SPLIT_FOLDER) if not "labels" in SIGNAL_DF_FILE]
test_signals_df = spark.read.format("parquet")\
    .load(test_signal_files)

In [33]:
test_signals_df.show()

+--------------------+--------------------+
|           subjectId|             signals|
+--------------------+--------------------+
|Anniepoo-20140308...|[0.0050354004, 0....|
+--------------------+--------------------+

